In [2]:
import os
import cv2
import numpy as np
import imgaug.augmenters as iaa
import matplotlib.pyplot as plt

# Define constants
data_path = "/Users/yanthraa/Desktop/PROJECT/NEW DATASET/archive"
target_size = (224, 224)
target_count = 1000  # Desired number of images per class
output_path = "/Users/yanthraa/Desktop/PROJECT/NEW DATASET/Augmented_Dataset"  # Output directory

# Ensure output directories exist
os.makedirs(output_path, exist_ok=True)
for class_name in ['Negatives', 'Positives']:
    os.makedirs(os.path.join(output_path, class_name), exist_ok=True)

# Define augmentation sequence
augmenters = iaa.Sequential([
    iaa.Fliplr(0.5),  # Horizontal flip
    iaa.Affine(rotate=(-15, 15)),  # Rotate between -15 and 15 degrees
    iaa.GaussianBlur(sigma=(0.0, 1.0)),  # Apply Gaussian blur
    iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # Add noise
    iaa.Multiply((0.8, 1.2))  # Brightness adjustments
])

# Load and preprocess data
images = []
labels = []

for class_name in ['Negatives', 'Positives']:
    class_path = os.path.join(data_path, class_name)
    class_images = []
    label = 0 if class_name == 'Negatives' else 1
    
    # Load original images
    for image_name in os.listdir(class_path):
        image_path = os.path.join(class_path, image_name)
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, target_size)
        img = img.astype(np.float32) / 255.0
        class_images.append(img)
    
    # Augment images until target count is reached
    while len(class_images) < target_count:
        augmented_images = augmenters(images=class_images[:min(len(class_images), target_count - len(class_images))])
        class_images.extend(augmented_images)
    
    class_images = class_images[:target_count]  # Trim excess
    images.extend(class_images)
    labels.extend([label] * target_count)
    
    # Save augmented images
    for idx, img in enumerate(class_images):
        save_path = os.path.join(output_path, class_name, f"aug_{idx}.png")
        cv2.imwrite(save_path, (img * 255).astype(np.uint8))

# Convert to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Print class proportions
print("Class Proportions:")
print("Negatives:", np.sum(labels == 0), "Positives:", np.sum(labels == 1))


Class Proportions:
Negatives: 1000 Positives: 1000


In [3]:
from sklearn.model_selection import train_test_split

# Separate data for Negatives and Positives classes
Negatives_images = images[labels == 0][:500]
Positives_images = images[labels == 1][:500]
Negatives_labels = labels[labels == 0][:500]
Positives_labels = labels[labels == 1][:500]

# Concatenate the data back together
balanced_images = np.concatenate([Negatives_images, Positives_images])
balanced_labels = np.concatenate([Negatives_labels, Positives_labels])


# Split the data into training and testing sets
X_train_full, X_test, y_train_full, y_test = train_test_split(balanced_images, balanced_labels, test_size=0.2, random_state=42)

# Split the training data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.1, random_state=42)

#print the balanced data
print("Balanced Images:",balanced_images.shape)
print("Balanced labels:",balanced_labels.shape)

#print the data seperately of each class
print("Negatives_images:",Negatives_images.shape)
print("Positives_images:",Positives_images.shape)


# Print the shapes of the training and testing sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


Balanced Images: (1000, 224, 224)
Balanced labels: (1000,)
Negatives_images: (500, 224, 224)
Positives_images: (500, 224, 224)
X_train shape: (720, 224, 224)
X_test shape: (200, 224, 224)
y_train shape: (720,)
y_test shape: (200,)


In [4]:
# Create a combined training dataset to use with KFold
X_train_combined = np.concatenate([X_train, X_valid])
y_train_combined = np.concatenate([y_train, y_valid])

In [5]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install tf-keras

Note: you may need to restart the kernel to use updated packages.


In [7]:
# New import statements to add
from sklearn.metrics import matthews_corrcoef, cohen_kappa_score, balanced_accuracy_score
from sklearn.metrics import log_loss
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, confusion_matrix
import time
# Add these new imports
from sklearn.metrics import matthews_corrcoef, cohen_kappa_score, balanced_accuracy_score, log_loss


In [21]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, confusion_matrix, matthews_corrcoef, balanced_accuracy_score, cohen_kappa_score
import time
import os
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Enable mixed precision training for faster computation
try:
    tf.keras.mixed_precision.set_global_policy('mixed_float16')
    print("Mixed precision training enabled")
except:
    print("Could not enable mixed precision training")
    
# Set TensorFlow to use the fastest possible algorithms
tf.config.optimizer.set_jit(True)  # Enable XLA compilation
print("XLA JIT compilation enabled")

# Set memory growth to avoid OOM errors
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)

class ClassTokenLayer(layers.Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        
    def build(self, input_shape):
        self.class_token = self.add_weight(
            shape=(1, 1, self.embed_dim),
            initializer="random_normal",
            trainable=True,
            name="class_token"
        )
    
    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        class_tokens = tf.repeat(self.class_token, batch_size, axis=0)
        return tf.concat([class_tokens, inputs], axis=1)
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1] + 1, input_shape[2])

class PositionEmbedding(layers.Layer):
    def __init__(self, num_patches, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.num_patches = num_patches
        self.embed_dim = embed_dim
        self.position_embedding = layers.Embedding(
            input_dim=num_patches + 1,
            output_dim=embed_dim
        )
        
    def call(self, inputs):
        positions = tf.range(start=0, limit=tf.shape(inputs)[1], delta=1)
        embedded_positions = self.position_embedding(positions)
        return inputs + embedded_positions
    
    def compute_output_shape(self, input_shape):
        return input_shape

# Use the built-in MultiHeadAttention for better performance
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, num_heads, embed_dim, dropout=0., **kwargs):
        super().__init__(**kwargs)
        self.num_heads = num_heads
        self.embed_dim = embed_dim
        
        # Use built-in MultiHeadAttention for much better performance
        self.mha = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=embed_dim // num_heads,
            dropout=dropout
        )
        
    def call(self, inputs, training=None):
        # Much faster implementation using built-in layer
        return self.mha(inputs, inputs, training=training)

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, mlp_dim, dropout=0.1, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.mlp_dim = mlp_dim
        
        # Pre-norm architecture for better performance
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.attention = MultiHeadSelfAttention(num_heads, embed_dim, dropout)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        
        # GELU activation for better accuracy
        self.mlp = keras.Sequential([
            layers.Dense(mlp_dim, activation=tf.nn.gelu),
            layers.Dropout(dropout),
            layers.Dense(embed_dim),
            layers.Dropout(dropout)
        ])
        
    def call(self, inputs, training=None):
        # Pre-norm architecture (more stable training)
        attn_output = self.attention(self.layernorm1(inputs), training=training)
        x = inputs + attn_output
        
        mlp_output = self.mlp(self.layernorm2(x), training=training)
        return x + mlp_output

def create_lightweight_deit_model(input_shape=(224, 224, 3), patch_size=32, embed_dim=192, 
                     num_heads=3, mlp_dim=None, num_layers=4, dropout_rate=0.1):
    """
    Create a lightweight DeiT model that's much faster while maintaining accuracy.
    Uses larger patch size, smaller embedding dimension, and fewer transformer layers.
    """
    if mlp_dim is None:
        mlp_dim = embed_dim * 2  # Reduced MLP size for faster computation
    
    num_patches = (input_shape[0] // patch_size) * (input_shape[1] // patch_size)
    
    inputs = layers.Input(shape=input_shape)
    
    # Use efficient patch embedding with larger patches for faster processing
    x = layers.Conv2D(embed_dim, kernel_size=patch_size, strides=patch_size)(inputs)
    x = layers.Reshape((-1, embed_dim))(x)
    
    # Add class token
    x = ClassTokenLayer(embed_dim)(x)
    
    # Add position embedding
    x = PositionEmbedding(num_patches, embed_dim)(x)
    x = layers.Dropout(dropout_rate)(x)
    
    # Fewer transformer blocks for speed
    for _ in range(num_layers):
        x = TransformerBlock(embed_dim, num_heads, mlp_dim, dropout_rate)(x)
    
    # Efficient classification head
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    x = layers.Lambda(lambda x: x[:, 0])(x)  # Take class token
    
    # Simplified classifier head - smaller but effective
    x = layers.Dense(256, activation='relu')(x)  # Using ReLU for better speed
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    return keras.Model(inputs=inputs, outputs=outputs)

def train_with_cv(X_train_combined, y_train_combined, X_test, y_test, cv_values=[5, 10, 15, 20], 
              optimize_for_speed=True, use_tpu=False):
    """
    Train the model with different cross-validation settings
    """
    # Prepare output directory
    os.makedirs('outputresults', exist_ok=True)
    
    # Make sure data has the right shape and channels
    # Check if input is already multi-channel
    if len(X_train_combined.shape) < 4 or X_train_combined.shape[-1] != 3:
        # If not, expand to 3 channels
        if len(X_train_combined.shape) < 4:
            X_train_combined = np.expand_dims(X_train_combined, axis=-1)
        if X_train_combined.shape[-1] == 1:
            X_train_combined = np.repeat(X_train_combined, 3, axis=-1)
    
    # Check if test data is already multi-channel
    if len(X_test.shape) < 4 or X_test.shape[-1] != 3:
        # If not, expand to 3 channels
        if len(X_test.shape) < 4:
            X_test = np.expand_dims(X_test, axis=-1)
        if X_test.shape[-1] == 1:
            X_test = np.repeat(X_test, 3, axis=-1)
    
    # Fix extra dimensions if present (issue in the error message)
    if len(X_test.shape) > 4:
        X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 3)
    
    print("Training data shape after adding channels:", X_train_combined.shape)
    print("Test data shape after adding channels:", X_test.shape)
    
    # If using TPU
    if use_tpu:
        try:
            resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
            tf.config.experimental_connect_to_cluster(resolver)
            tf.tpu.experimental.initialize_tpu_system(resolver)
            strategy = tf.distribute.TPUStrategy(resolver)
            print("TPU detected and initialized!")
        except:
            strategy = tf.distribute.MirroredStrategy()
            print("No TPU detected, using MirroredStrategy")
    else:
        # For multi-GPU training if available
        strategy = tf.distribute.MirroredStrategy()
        print(f"Number of devices: {strategy.num_replicas_in_sync}")
    
    # Aggressive training settings for faster convergence
    callbacks = [
        EarlyStopping(
            monitor='val_accuracy' if optimize_for_speed else 'val_loss',
            patience=5,  # More aggressive early stopping
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_accuracy' if optimize_for_speed else 'val_loss',
            factor=0.2,  # More aggressive LR reduction
            patience=3,  # Reduce patience for faster adaptation
            min_lr=1e-6,
            verbose=1
        )
    ]
    
    all_metrics = []
    
    # Train with different CV settings
    for k in cv_values:
        print(f"\n=== Training with {k}-fold Cross Validation ===")
        kf = KFold(n_splits=k, shuffle=True, random_state=42)
        
        metrics_list = []
        
        for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_combined)):
            print(f'Fold {fold + 1}/{k}')
            
            # Split data
            X_train_fold = X_train_combined[train_idx]
            y_train_fold = y_train_combined[train_idx]
            X_val_fold = X_train_combined[val_idx]
            y_val_fold = y_train_combined[val_idx]
            
            print(f"Training fold shape: {X_train_fold.shape}")
            print(f"Validation fold shape: {X_val_fold.shape}")
            
            # Data augmentation for better generalization
            data_augmentation = keras.Sequential([
                layers.RandomFlip("horizontal"),
                layers.RandomRotation(0.1),
                layers.RandomZoom(0.1),
            ])
            
            # Create much faster lightweight model
            model = create_lightweight_deit_model(
                patch_size=32,    # Larger patches = fewer tokens = much faster training
                embed_dim=192,    # Smaller embedding dimension for efficiency
                num_heads=3,      # Fewer attention heads
                num_layers=4,     # Greatly reduced number of layers for speed
                dropout_rate=0.2  # Keep good regularization
            )
            
            # Simple but effective optimizer
            optimizer = keras.optimizers.Adam(
                learning_rate=5e-4  # Moderate learning rate for stability
            )
            
            # Use label smoothing for better accuracy
            model.compile(
                optimizer=optimizer,
                loss=keras.losses.BinaryCrossentropy(label_smoothing=0.1),  # Label smoothing helps accuracy
                metrics=['accuracy', 'Precision', 'Recall', 'AUC']
            )
            
            # Add model checkpoint for this fold
            fold_checkpoint = ModelCheckpoint(
                f'outputresults/model_cv{k}_fold{fold+1}.h5',
                monitor='val_accuracy',  # Focus on accuracy, not just loss
                save_best_only=True,
                verbose=1
            )
            
            # Create a custom callback to speed up training if accuracy is already good
            early_accuracy = EarlyStopping(
                monitor='val_accuracy',
                min_delta=0.005,
                patience=5,
                baseline=0.85,  # Stop if we reach good accuracy
                verbose=1
            )
            
            fold_callbacks = callbacks + [fold_checkpoint, early_accuracy]
            
            # Train model with much faster settings
            start_time = time.time()
            history = model.fit(
                data_augmentation(X_train_fold), 
                y_train_fold,
                validation_data=(X_val_fold, y_val_fold),
                epochs=50,        # Fewer maximum epochs
                batch_size=128,   # Much larger batch size for speed
                callbacks=fold_callbacks,
                verbose=1
            )
            training_time = time.time() - start_time
            
            # Load best model weights for evaluation
            model.load_weights(f'outputresults/model_cv{k}_fold{fold+1}.h5')
            
            # Evaluate model
            test_results = model.evaluate(X_test, y_test, verbose=0)
            y_pred = model.predict(X_test)
            y_pred_binary = (y_pred > 0.5).astype(int)
            
            # Calculate confusion matrix metrics
            try:
                tn, fp, fn, tp = confusion_matrix(y_test, y_pred_binary).ravel()
                
                # Calculate additional metrics
                specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
                mcc = matthews_corrcoef(y_test, y_pred_binary)
                # Calculate log loss (cross-entropy)
                log_loss = -np.mean(y_test * np.log(np.clip(y_pred, 1e-15, 1.0)) + 
                                   (1 - y_test) * np.log(np.clip(1 - y_pred, 1e-15, 1.0)))
                # Calculate geometric mean score
                sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0  # same as recall
                g_mean = np.sqrt(sensitivity * specificity) if (sensitivity * specificity) > 0 else 0
                # Calculate Youden's J statistic (J = sensitivity + specificity - 1)
                youdens_j = sensitivity + specificity - 1
                balanced_acc = balanced_accuracy_score(y_test, y_pred_binary)
                kappa = cohen_kappa_score(y_test, y_pred_binary)
                
                # Store metrics
                fold_metrics = {
                    'CV': k,
                    'Fold': fold + 1,
                    'Test_Accuracy': test_results[1],
                    'Train_Accuracy': history.history['accuracy'][-1],
                    'Val_Accuracy': history.history['val_accuracy'][-1],
                    'Precision': test_results[2],
                    'Recall': test_results[3],
                    'AUC_ROC': test_results[4],
                    'F1_Score': f1_score(y_test, y_pred_binary),
                    'Training_Loss': history.history['loss'][-1],
                    'Val_Loss': history.history['val_loss'][-1],
                    'Testing_Loss': test_results[0],
                    'Time_Taken': training_time,
                    'TP': tp,
                    'TN': tn,
                    'FP': fp,
                    'FN': fn,
                    'Specificity': specificity,
                    'MCC': mcc,
                    'Log_Loss': log_loss,
                    'G_Mean': g_mean,
                    'Youdens_J': youdens_j,
                    'Balanced_Accuracy': balanced_acc,
                    'Cohens_Kappa': kappa,
                    'Epochs_Trained': len(history.history['loss']),
                    'Final_Learning_Rate': float(tf.keras.backend.get_value(model.optimizer.learning_rate))
                }
                metrics_list.append(fold_metrics)
            except Exception as e:
                print(f"Error calculating metrics: {e}")
        
        # Save metrics to CSV for this CV setting
        if metrics_list:
            metrics_df = pd.DataFrame(metrics_list)
            metrics_df.to_csv(f'outputresults/deit_metrics_cv{k}.csv', index=False)
            print(f"DeiT training completed for CV={k} and metrics saved to deit_metrics_cv{k}.csv")
            
            # Compute and print average metrics
            avg_metrics = {
                'CV': k,
                'Avg_Test_Accuracy': metrics_df['Test_Accuracy'].mean(),
                'Avg_Training_Loss': metrics_df['Training_Loss'].mean(),
                'Avg_Testing_Loss': metrics_df['Testing_Loss'].mean(),
                'Avg_F1_Score': metrics_df['F1_Score'].mean(),
                'Avg_Training_Time': metrics_df['Time_Taken'].mean(),
                'Avg_Epochs': metrics_df['Epochs_Trained'].mean()
            }
            print("\nAverage metrics:")
            for key, value in avg_metrics.items():
                print(f"{key}: {value:.4f}")
                
            all_metrics.append(avg_metrics)
    
    # Save summary of all CV settings
    if all_metrics:
        summary_df = pd.DataFrame(all_metrics)
        summary_df.to_csv('outputresults/deit_cv_summary.csv', index=False)
        print("\nCV summary saved to deit_cv_summary.csv")
        
    return all_metrics

# Example usage:
if __name__ == "__main__":
    # Example of how to use the function with the optimization flags
    train_with_cv(
        X_train_combined, 
        y_train_combined, 
        X_test, 
        y_test, 
        cv_values=[5, 10, 15, 20],
        optimize_for_speed=True,  # Set to True for fastest training
        use_tpu=False             # Set to True if TPU is available
    )
    print("Ready to run with your data. Set optimize_for_speed=True for much faster training.")

Mixed precision training enabled
XLA JIT compilation enabled
Training data shape after adding channels: (800, 224, 224, 3)
Test data shape after adding channels: (200, 224, 224, 3)
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


Number of devices: 1

=== Training with 5-fold Cross Validation ===
Fold 1/5
Training fold shape: (640, 224, 224, 3)
Validation fold shape: (160, 224, 224, 3)
Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - AUC: 0.4541 - Precision: 0.4280 - Recall: 0.4094 - accuracy: 0.4530 - loss: 0.9465
Epoch 1: val_accuracy improved from -inf to 0.51875, saving model to outputresults/model_cv5_fold1.h5


5/5 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - AUC: 0.4533 - Precision: 0.4286 - Recall: 0.4072 - accuracy: 0.4525 - loss: 0.9538 - val_AUC: 0.7592 - val_Precision: 0.5188 - val_Recall: 1.0000 - val_accuracy: 0.5188 - val_loss: 0.8387 - learning_rate: 5.0000e-04
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - AUC: 0.4941 - Precision: 0.4888 - Recall: 0.7014 - accuracy: 0.4979 - loss: 0.7693
Epoch 2: val_accuracy did not improve from 0.51875
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - AUC: 0.4951 - Precision: 0.4869 - Recall: 0.6569 - accuracy: 0.4967 - loss: 0.7675 - val_AUC: 0.7798 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00 - val_accuracy: 0.4812 - val_loss: 0.7258 - learning_rate: 5.0000e-04
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - AUC: 0.5982 - Precision: 0.6119 - Recall: 0.4100 - accuracy: 0.5679 - loss: 0.6956
Epoch 3: val_accuracy improved from 0.51875 to 0.71875, saving model to outputresults/model_cv5_fold1.h5


5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - AUC: 0.6011 - Precision: 0.6055 - Recall: 0.4455 - accuracy: 0.5715 - loss: 0.6937 - val_AUC: 0.7523 - val_Precision: 0.6863 - val_Recall: 0.8434 - val_accuracy: 0.7188 - val_loss: 0.6072 - learning_rate: 5.0000e-04
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - AUC: 0.6740 - Precision: 0.6489 - Recall: 0.8425 - accuracy: 0.6912 - loss: 0.6399
Epoch 4: val_accuracy improved from 0.71875 to 0.74375, saving model to outputresults/model_cv5_fold1.h5


...

6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - AUC: 0.6559 - Precision: 0.6152 - Recall: 0.3905 - accuracy: 0.5909 - loss: 0.6683 - val_AUC: 0.7747 - val_Precision: 0.7000 - val_Recall: 0.8750 - val_accuracy: 0.7000 - val_loss: 0.5881 - learning_rate: 5.0000e-04
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - AUC: 0.6856 - Precision: 0.6361 - Recall: 0.8630 - accuracy: 0.6813 - loss: 0.6545
Epoch 5: val_accuracy improved from 0.70000 to 0.77500, saving model to outputresults/model_cv20_fold19.h5


6/6 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - AUC: 0.6874 - Precision: 0.6348 - Recall: 0.8588 - accuracy: 0.6806 - loss: 0.6531 - val_AUC: 0.7786 - val_Precision: 0.8000 - val_Recall: 0.8333 - val_accuracy: 0.7750 - val_loss: 0.6781 - learning_rate: 5.0000e-04
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - AUC: 0.7516 - Precision: 0.6919 - Recall: 0.7427 - accuracy: 0.7048 - loss: 0.6141
Epoch 6: val_accuracy improved from 0.77500 to 0.80000, saving model to outputresults/model_cv20_fold19.h5


6/6 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - AUC: 0.7477 - Precision: 0.6871 - Recall: 0.7491 - accuracy: 0.7035 - loss: 0.6155 - val_AUC: 0.7799 - val_Precision: 0.8077 - val_Recall: 0.8750 - val_accuracy: 0.8000 - val_loss: 0.5768 - learning_rate: 5.0000e-04
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - AUC: 0.7002 - Precision: 0.6661 - Recall: 0.8723 - accuracy: 0.7172 - loss: 0.6220
Epoch 7: val_accuracy did not improve from 0.80000
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - AUC: 0.7032 - Precision: 0.6665 - Recall: 0.8694 - accuracy: 0.7178 - loss: 0.6201 - val_AUC: 0.7812 - val_Precision: 0.8000 - val_Recall: 0.8333 - val_accuracy: 0.7750 - val_loss: 0.6224 - learning_rate: 5.0000e-04
Restoring model weights from the end of the best epoch: 6.
Epoch 7: early stopping
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step
Fold 20/20
Training fold shape: (760, 224, 224, 3)
Validation fold shape: (40, 224, 224, 3)
Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - AUC: 0.4997 - Precision: 0.4681 - Recall: 0.59

6/6 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - AUC: 0.4996 - Precision: 0.4706 - Recall: 0.5860 - accuracy: 0.4821 - loss: 0.8896 - val_AUC: 0.7945 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00 - val_accuracy: 0.5250 - val_loss: 0.7077 - learning_rate: 5.0000e-04
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - AUC: 0.5238 - Precision: 0.5482 - Recall: 0.4872 - accuracy: 0.5357 - loss: 0.7267
Epoch 2: val_accuracy did not improve from 0.52500
6/6 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - AUC: 0.5248 - Precision: 0.5452 - Recall: 0.5059 - accuracy: 0.5357 - loss: 0.7265 - val_AUC: 0.8396 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00 - val_accuracy: 0.5250 - val_loss: 0.6358 - learning_rate: 5.0000e-04
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - AUC: 0.5636 - Precision: 0.5262 - Recall: 0.3570 - accuracy: 0.5246 - loss: 0.7151
Epoch 3: val_accuracy improved from 0.52500 to 0.75000, saving model to outputresults/model_cv20_fold20.h5


6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - AUC: 0.5711 - Precision: 0.5345 - Recall: 0.3780 - accuracy: 0.5320 - loss: 0.7112 - val_AUC: 0.8434 - val_Precision: 0.6667 - val_Recall: 0.9474 - val_accuracy: 0.7500 - val_loss: 0.5582 - learning_rate: 5.0000e-04
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - AUC: 0.7183 - Precision: 0.6348 - Recall: 0.8029 - accuracy: 0.6765 - loss: 0.6363
Epoch 4: val_accuracy did not improve from 0.75000
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - AUC: 0.7195 - Precision: 0.6375 - Recall: 0.8011 - accuracy: 0.6782 - loss: 0.6354 - val_AUC: 0.8496 - val_Precision: 0.6957 - val_Recall: 0.8421 - val_accuracy: 0.7500 - val_loss: 0.5479 - learning_rate: 5.0000e-04
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - AUC: 0.7050 - Precision: 0.6370 - Recall: 0.8238 - accuracy: 0.6723 - loss: 0.6392
Epoch 5: val_accuracy did not improve from 0.75000
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - AUC: 0.7069 - Precision: 0.6389 - Recall: 0.8190 - accuracy: 0.6742 - loss: 0.6372 - val

In [43]:
import pandas as pd
# List of files in the desired order
files = [
    'outputresults/deit_metrics_cv5.csv',
    'outputresults/deit_metrics_cv10.csv',  # Added comma here
    'outputresults/deit_metrics_cv15.csv',
    'outputresults/deit_metrics_cv20.csv'   # Added comma here
]
# Function to append CSV files
def append_csv_files(file_list, output_file='outputresults/deit_metrics111.csv'):
    # Initialize an empty DataFrame to store the combined data
    combined_df = pd.DataFrame()
    
    # Read and append each file
    for file in file_list:
        try:
            df = pd.read_csv(file)
            combined_df = pd.concat([combined_df, df], ignore_index=True)
            print(f"Successfully appended: {file}")
        except Exception as e:
            print(f"Error appending {file}: {e}")
    # Save the combined DataFrame to a new CSV file
    combined_df.to_csv(output_file, index=False)
    row_count = len(combined_df)
    print(f"Combined data saved to {output_file}")
    print(f"Total number of rows in {output_file}: {row_count}")
# Execute the function
append_csv_files(files)

Successfully appended: outputresults/deit_metrics_cv5.csv
Successfully appended: outputresults/deit_metrics_cv10.csv
Successfully appended: outputresults/deit_metrics_cv15.csv
Successfully appended: outputresults/deit_metrics_cv20.csv
Combined data saved to outputresults/deit_metrics111.csv
Total number of rows in outputresults/deit_metrics111.csv: 50


In [7]:
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import KFold
# from sklearn.metrics import f1_score, confusion_matrix
# import time

# class ClassTokenLayer(layers.Layer):
#     def __init__(self, embed_dim, **kwargs):
#         super().__init__(**kwargs)
#         self.embed_dim = embed_dim
        
#     def build(self, input_shape):
#         self.class_token = self.add_weight(
#             shape=(1, 1, self.embed_dim),
#             initializer="random_normal",
#             trainable=True,
#             name="class_token"
#         )
    
#     def call(self, inputs):
#         batch_size = tf.shape(inputs)[0]
#         class_tokens = tf.repeat(self.class_token, batch_size, axis=0)
#         return tf.concat([class_tokens, inputs], axis=1)
    
#     def compute_output_shape(self, input_shape):
#         return (input_shape[0], input_shape[1] + 1, input_shape[2])

# class PositionEmbedding(layers.Layer):
#     def __init__(self, num_patches, embed_dim, **kwargs):
#         super().__init__(**kwargs)
#         self.num_patches = num_patches
#         self.embed_dim = embed_dim
#         self.position_embedding = layers.Embedding(
#             input_dim=num_patches + 1,
#             output_dim=embed_dim
#         )
        
#     def call(self, inputs):
#         positions = tf.range(start=0, limit=tf.shape(inputs)[1], delta=1)
#         embedded_positions = self.position_embedding(positions)
#         return inputs + embedded_positions
    
#     def compute_output_shape(self, input_shape):
#         return input_shape

# class MultiHeadSelfAttention(layers.Layer):
#     def __init__(self, num_heads, embed_dim, dropout=0., **kwargs):
#         super().__init__(**kwargs)
#         self.num_heads = num_heads
#         self.embed_dim = embed_dim
#         self.head_dim = embed_dim // num_heads
#         self.scale = self.head_dim ** -0.5
        
#         self.qkv_dense = layers.Dense(3 * embed_dim)
#         self.out_dense = layers.Dense(embed_dim)
#         self.dropout = layers.Dropout(dropout)
        
#     def build(self, input_shape):
#         super().build(input_shape)
    
#     def call(self, inputs, training=None):
#         batch_size = tf.shape(inputs)[0]
#         seq_len = tf.shape(inputs)[1]
        
#         qkv = self.qkv_dense(inputs)
#         qkv = tf.reshape(qkv, (batch_size, seq_len, 3, self.num_heads, self.head_dim))
#         qkv = tf.transpose(qkv, perm=[2, 0, 3, 1, 4])
#         q, k, v = tf.unstack(qkv, axis=0)
        
#         attention = tf.matmul(q, k, transpose_b=True) * self.scale
#         attention = tf.nn.softmax(attention, axis=-1)
#         attention = self.dropout(attention, training=training)
        
#         x = tf.matmul(attention, v)
#         x = tf.transpose(x, perm=[0, 2, 1, 3])
#         x = tf.reshape(x, (batch_size, seq_len, self.embed_dim))
#         x = self.out_dense(x)
#         return x

# class TransformerBlock(layers.Layer):
#     def __init__(self, embed_dim, num_heads, mlp_dim, dropout=0.1, **kwargs):
#         super().__init__(**kwargs)
#         self.embed_dim = embed_dim
#         self.num_heads = num_heads
#         self.mlp_dim = mlp_dim
        
#         self.attention = MultiHeadSelfAttention(num_heads, embed_dim, dropout)
#         self.mlp = keras.Sequential([
#             layers.Dense(mlp_dim, activation=tf.nn.gelu),
#             layers.Dropout(dropout),
#             layers.Dense(embed_dim),
#             layers.Dropout(dropout)
#         ])
#         self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
#         self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
#         self.dropout = layers.Dropout(dropout)
        
#     def build(self, input_shape):
#         super().build(input_shape)
    
#     def call(self, inputs, training=None):
#         x1 = self.layernorm1(inputs)
#         attention_output = self.attention(x1, training=training)
#         attention_output = self.dropout(attention_output, training=training)
#         x2 = inputs + attention_output
        
#         x3 = self.layernorm2(x2)
#         x3 = self.mlp(x3, training=training)
#         return x2 + x3

# def create_deit_model(input_shape=(224, 224, 3), patch_size=16):
#     embed_dim = 384
#     num_heads = 6
#     mlp_dim = embed_dim * 4
#     num_layers = 12
#     dropout_rate = 0.1
    
#     num_patches = (input_shape[0] // patch_size) * (input_shape[1] // patch_size)
    
#     inputs = layers.Input(shape=input_shape)
    
#     # Patch embedding
#     x = layers.Conv2D(embed_dim, patch_size, patch_size)(inputs)
#     x = layers.Reshape((-1, embed_dim))(x)
    
#     # Add class token
#     x = ClassTokenLayer(embed_dim)(x)
    
#     # Add position embedding
#     x = PositionEmbedding(num_patches, embed_dim)(x)
    
#     # Transformer blocks
#     for _ in range(num_layers):
#         x = TransformerBlock(embed_dim, num_heads, mlp_dim, dropout_rate)(x)
    
#     # Layer norm and head
#     x = layers.LayerNormalization(epsilon=1e-6)(x)
#     x = layers.Lambda(lambda x: x[:, 0])(x)  # Take class token
#     x = layers.Dense(512, activation='relu')(x)
#     x = layers.Dropout(dropout_rate)(x)
#     outputs = layers.Dense(1, activation='sigmoid')(x)
    
#     return keras.Model(inputs=inputs, outputs=outputs)

# metrics_list = []
# k = 5  # number of folds
# kf = KFold(n_splits=k, shuffle=True, random_state=42)

# # Add channel dimension to the data
# X_train_combined = np.expand_dims(X_train_combined, axis=-1)
# X_test = np.expand_dims(X_test, axis=-1)

# # Repeat the single channel to make it 3 channels (since model expects RGB)
# X_train_combined = np.repeat(X_train_combined, 3, axis=-1)
# X_test = np.repeat(X_test, 3, axis=-1)

# print("Training data shape after adding channels:", X_train_combined.shape)
# print("Test data shape after adding channels:", X_test.shape)

# for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_combined)):
#     print(f'Fold {fold + 1}/{k}')
    
#     # Split data
#     X_train_fold = X_train_combined[train_idx]
#     y_train_fold = y_train_combined[train_idx]
#     X_val_fold = X_train_combined[val_idx]
#     y_val_fold = y_train_combined[val_idx]
    
#     print(f"Training fold shape: {X_train_fold.shape}")
#     print(f"Validation fold shape: {X_val_fold.shape}")
    
#     # Create and compile model
#     model = create_deit_model()
#     model.compile(
#         optimizer=keras.optimizers.Adam(learning_rate=0.001),
#         loss='binary_crossentropy',
#         metrics=['accuracy', 'Precision', 'Recall', 'AUC']
#     )
    
#     # Train model
#     start_time = time.time()
#     history = model.fit(
#         X_train_fold, 
#         y_train_fold,
#         validation_data=(X_val_fold, y_val_fold),
#         epochs=50,
#         batch_size=32,
#         verbose=1
#     )
#     training_time = time.time() - start_time
    
#     # Evaluate model
#     test_results = model.evaluate(X_test, y_test)
#     y_pred = model.predict(X_test)
#     y_pred_binary = (y_pred > 0.5).astype(int)

#     # Modify the metrics calculation section in your fold loop as follows:
#     # Calculate confusion matrix metrics
#     tn, fp, fn, tp = confusion_matrix(y_test, y_pred_binary).ravel()
    
#     # Calculate additional metrics
#     specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
#     mcc = matthews_corrcoef(y_test, y_pred_binary)
#     # Calculate log loss (cross-entropy)
#     log_loss = -np.mean(y_test * np.log(np.clip(y_pred, 1e-15, 1.0)) + 
#                         (1 - y_test) * np.log(np.clip(1 - y_pred, 1e-15, 1.0)))
#     # Calculate geometric mean score
#     sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0  # same as recall
#     g_mean = np.sqrt(sensitivity * specificity) if (sensitivity * specificity) > 0 else 0
#     # Calculate Youden's J statistic (J = sensitivity + specificity - 1)
#     youdens_j = sensitivity + specificity - 1
#     balanced_acc = balanced_accuracy_score(y_test, y_pred_binary)
#     kappa = cohen_kappa_score(y_test, y_pred_binary)
    
#     # Store metrics (existing and new)
#     fold_metrics = {
#         'Fold': fold + 1,
#         'K': k,
#         'Test_Accuracy': test_results[1],
#         'Train_Accuracy': history.history['accuracy'][-1],
#         'Precision': test_results[2],
#         'Recall': test_results[3],
#         'AUC_ROC': test_results[4],
#         'F1_Score': f1_score(y_test, y_pred_binary),
#         'Training_Loss': history.history['loss'][-1],
#         'Testing_Loss': test_results[0],
#         'Time_Taken': training_time,
#         'TP': tp,
#         'TN': tn,
#         'FP': fp,
#         'FN': fn,
#         # New metrics
#         'Specificity': specificity,
#         'MCC': mcc,
#         'Log_Loss': log_loss,
#         'G_Mean': g_mean,
#         'Youdens_J': youdens_j,
#         'Balanced_Accuracy': balanced_acc,
#         'Cohens_Kappa': kappa
#     }
#     metrics_list.append(fold_metrics)

# # Save metrics to CSV (unchanged)
# metrics_df = pd.DataFrame(metrics_list)
# metrics_df.to_csv('outputresults/deit_metrics111.csv', index=False)
# print("DeiT training completed and metrics saved to deit_metrics.csv")


Training data shape after adding channels: (800, 224, 224, 3)
Test data shape after adding channels: (200, 224, 224, 3)
Fold 1/5
Training fold shape: (640, 224, 224, 3)
Validation fold shape: (160, 224, 224, 3)
Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 245s 12s/step - AUC: 0.5457 - Precision: 0.5053 - Recall: 0.4217 - accuracy: 0.5504 - loss: 1.3106 - val_AUC: 0.7741 - val_Precision: 0.7500 - val_Recall: 0.7952 - val_accuracy: 0.7563 - val_loss: 0.5897
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 232s 12s/step - AUC: 0.7771 - Precision: 0.7096 - Recall: 0.8464 - accuracy: 0.7563 - loss: 0.5427 - val_AUC: 0.7813 - val_Precision: 0.7500 - val_Recall: 0.7952 - val_accuracy: 0.7563 - val_loss: 0.5581
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 232s 12s/step - AUC: 0.7959 - Precision: 0.7241 - Recall: 0.8794 - accuracy: 0.7848 - loss: 0.5435 - val_AUC: 0.7876 - val_Precision: 0.7444 - val_Recall: 0.8072 - val_accuracy: 0.7563 - val_loss: 0.5691
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 231s 12s/step - AUC: 0.81

In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import (
    f1_score, confusion_matrix, log_loss, matthews_corrcoef, 
    balanced_accuracy_score, cohen_kappa_score
)
import time
import gc
import os

# Enable memory growth to prevent GPU memory allocation issues
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
    print("GPU memory growth enabled")

# Simplified Squeeze-and-Excitation block
class SE(layers.Layer):
    def __init__(self, channels, ratio=16, **kwargs):
        super().__init__(**kwargs)
        self.channels = channels
        self.ratio = ratio
        self.squeeze = layers.GlobalAveragePooling2D()
        self.excitation = keras.Sequential([
            layers.Dense(max(channels // ratio, 8), activation='relu'),
            layers.Dense(channels, activation='sigmoid'),
            layers.Reshape((1, 1, channels))
        ])
        
    def call(self, inputs):
        x = self.squeeze(inputs)
        x = self.excitation(x)
        return inputs * x

# Optimized MBConv block
class MBConv(layers.Layer):
    def __init__(self, out_channels, stride, expand_ratio, **kwargs):
        super().__init__(**kwargs)
        self.out_channels = out_channels
        self.stride = stride
        self.expand_ratio = expand_ratio
        
    def build(self, input_shape):
        in_channels = input_shape[-1]
        hidden_dim = int(in_channels * self.expand_ratio)
        
        self.expand = None
        if self.expand_ratio != 1:
            self.expand = keras.Sequential([
                layers.Conv2D(hidden_dim, 1, padding='same', use_bias=False),
                layers.BatchNormalization(),
                layers.Activation('swish')
            ])
            
        self.depthwise = keras.Sequential([
            layers.DepthwiseConv2D(3, self.stride, padding='same', use_bias=False),
            layers.BatchNormalization(),
            layers.Activation('swish')
        ])
        
        self.se = SE(hidden_dim)
        
        self.project = keras.Sequential([
            layers.Conv2D(self.out_channels, 1, padding='same', use_bias=False),
            layers.BatchNormalization()
        ])
        
        self.shortcut = input_shape[-1] == self.out_channels and self.stride == 1
        
    def call(self, inputs):
        x = inputs
        if self.expand is not None:
            x = self.expand(x)
        x = self.depthwise(x)
        x = self.se(x)
        x = self.project(x)
        
        if self.shortcut:
            return inputs + x
        return x

# More efficient transformer implementation
class TransformerBlock(layers.Layer):
    def __init__(self, dim, num_heads, mlp_ratio=4, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim
        self.num_heads = num_heads
        self.mlp_ratio = mlp_ratio
        self.norm1 = layers.LayerNormalization(epsilon=1e-6)
        self.attn = layers.MultiHeadAttention(
            num_heads=num_heads, 
            key_dim=dim // num_heads
        )
        self.norm2 = layers.LayerNormalization(epsilon=1e-6)
        self.mlp = keras.Sequential([
            layers.Dense(int(dim * mlp_ratio), activation='gelu'),
            layers.Dropout(0.1),  # Add dropout for regularization
            layers.Dense(dim)
        ])
        self.dropout = layers.Dropout(0.1)
        
    def call(self, inputs, training=False):
        # Apply layer normalization before attention (pre-norm formulation)
        x1 = self.norm1(inputs)
        
        # Reshape for multi-head attention if needed
        batch_size = tf.shape(inputs)[0]
        height = tf.shape(inputs)[1]
        width = tf.shape(inputs)[2]
        
        # Reshape to sequence format for attention
        x1_flat = tf.reshape(x1, [batch_size, height * width, self.dim])
        
        # Apply attention
        attention_output = self.attn(x1_flat, x1_flat)
        attention_output = self.dropout(attention_output, training=training)
        
        # Reshape back to spatial format
        attention_output = tf.reshape(attention_output, [batch_size, height, width, self.dim])
        
        # First residual connection
        x2 = inputs + attention_output
        
        # Second normalization
        x3 = self.norm2(x2)
        
        # MLP block
        mlp_output = self.mlp(x3)
        mlp_output = self.dropout(mlp_output, training=training)
        
        # Second residual connection
        return x2 + mlp_output

# Create optimized CoAtNet model with reduced complexity
def create_coatnet_model(input_shape=(28, 28, 3), num_classes=1):
    # Reduced dimensions for faster training
    dims = [16, 32, 64, 128]  # Significantly reduced from original
    num_blocks = [1, 1, 2, 2]  # Reduced number of blocks
    
    inputs = layers.Input(shape=input_shape)
    
    # Stage 0: Stem
    x = layers.Conv2D(dims[0], 3, 2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('swish')(x)
    
    # Stage 1: MBConv
    for _ in range(num_blocks[0]):
        x = MBConv(dims[0], stride=1, expand_ratio=2)(x)  # Reduced expand ratio
    
    # Stage 2: MBConv + Transformer
    x = MBConv(dims[1], stride=2, expand_ratio=2)(x)
    for _ in range(num_blocks[1] - 1):
        x = TransformerBlock(dims[1], num_heads=4)(x)  # Reduced heads
    
    # Stage 3: Transformer
    x = layers.Conv2D(dims[2], 1, 2)(x)
    for _ in range(num_blocks[2]):
        x = TransformerBlock(dims[2], num_heads=4)(x)
    
    # Stage 4: Transformer
    x = layers.Conv2D(dims[3], 1, 2)(x)
    for _ in range(num_blocks[3]):
        x = TransformerBlock(dims[3], num_heads=8)(x)
    
    # Head
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)  # Reduced from 512
    x = layers.Dropout(0.3)(x)  # Increased dropout for better regularization
    
    # Output layer
    if num_classes == 1:
        outputs = layers.Dense(1, activation='sigmoid')(x)
    else:
        outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    return keras.Model(inputs=inputs, outputs=outputs)

# Function to calculate additional metrics
def calculate_additional_metrics(y_true, y_pred, y_pred_proba=None):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    # Calculate metrics
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    mcc = matthews_corrcoef(y_true, y_pred)
    balanced_acc = balanced_accuracy_score(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)
    
    # Geometric mean
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    g_mean = np.sqrt(sensitivity * specificity) if (sensitivity * specificity) > 0 else 0
    
    # Youden's J statistic
    youdens_j = sensitivity + specificity - 1
    
    # Log loss (if probabilities provided)
    if y_pred_proba is not None:
        log_loss_value = log_loss(y_true, y_pred_proba)
    else:
        log_loss_value = None
    
    return {
        'Specificity': specificity,
        'MCC': mcc,
        'Log_Loss': log_loss_value,
        'G_Mean': g_mean,
        'Youdens_J': youdens_j,
        'Balanced_Accuracy': balanced_acc,
        'Cohens_Kappa': kappa
    }

# Main training function
def train_coatnet(X_train_full, y_train_full, X_test, y_test, k_folds=[5, 10], epochs=50):
    # Create metrics dataframe to store all results
    all_metrics = []
    
    # Define callbacks for training
    callbacks = [
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True,
            min_delta=0.001
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            min_lr=1e-6
        )
    ]
    
    # Process each K-fold setting
    for k in k_folds:
        print(f"\n=== Starting {k}-fold cross-validation ===")
        
        # Initialize K-fold cross-validation
        kf = KFold(n_splits=k, shuffle=True, random_state=42)
        
        # Store metrics for each fold
        fold_metrics = []
        
        # Run K-fold cross-validation
        for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_full)):
            print(f'Fold {fold + 1}/{k}')
            
            # Split data
            X_train_fold = X_train_full[train_idx]
            y_train_fold = y_train_full[train_idx]
            X_val_fold = X_train_full[val_idx]
            y_val_fold = y_train_full[val_idx]
            
            print(f"Training fold shape: {X_train_fold.shape}")
            print(f"Validation fold shape: {X_val_fold.shape}")
            
            # Create and compile model
            model = create_coatnet_model(input_shape=X_train_full.shape[1:])
            
            # Use mixed precision for faster training
            if physical_devices:
                optimizer = keras.mixed_precision.LossScaleOptimizer(
                    keras.optimizers.Adam(learning_rate=1e-3)
                )
            else:
                optimizer = keras.optimizers.Adam(learning_rate=1e-3)
                
            model.compile(
                optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=['accuracy', 'Precision', 'Recall', 'AUC']
            )
            
            # Train model
            start_time = time.time()
            history = model.fit(
                X_train_fold, 
                y_train_fold,
                validation_data=(X_val_fold, y_val_fold),
                epochs=epochs,
                batch_size=64,  # Larger batch size for faster training
                callbacks=callbacks,
                verbose=1
            )
            training_time = time.time() - start_time
            
            # Evaluate model
            test_results = model.evaluate(X_test, y_test, verbose=0)
            y_pred_proba = model.predict(X_test, verbose=0)
            y_pred_binary = (y_pred_proba > 0.5).astype(int)
            
            # Calculate confusion matrix metrics
            tn, fp, fn, tp = confusion_matrix(y_test, y_pred_binary).ravel()
            
            # Calculate additional metrics
            additional_metrics = calculate_additional_metrics(
                y_test, y_pred_binary, y_pred_proba
            )
            
            # Store metrics
            metrics = {
                'Fold': fold + 1,
                'K': k,
                'Test_Accuracy': test_results[1],
                'Train_Accuracy': history.history['accuracy'][-1],
                'Precision': test_results[2],
                'Recall': test_results[3],
                'AUC_ROC': test_results[4],
                'F1_Score': f1_score(y_test, y_pred_binary),
                'Training_Loss': history.history['loss'][-1],
                'Testing_Loss': test_results[0],
                'Time_Taken': training_time,
                'TP': tp,
                'TN': tn,
                'FP': fp,
                'FN': fn,
                'Epochs_Run': len(history.history['loss']),
                **additional_metrics
            }
            
            fold_metrics.append(metrics)
            
            # Clear memory
            del model
            del history
            keras.backend.clear_session()
            gc.collect()
        
        # Add to all metrics
        all_metrics.extend(fold_metrics)
    
    # Convert to DataFrame and save
    metrics_df = pd.DataFrame(all_metrics)
    metrics_df.to_csv('outputresults/optimized_coatnet_metrics111.csv', index=False)
    print("CoAtNet training completed and metrics saved to optimized_coatnet_metrics.csv")
    
    return metrics_df

# Main execution
if __name__ == "__main__":
    # Check if data exists (assuming X_train_full, y_train_full, X_test, y_test)
    # You need to make sure these variables are defined before running
    
    # Convert data to correct format
    if len(X_train_full.shape) == 3:  # If grayscale images
        # Add channel dimension and repeat to make it 3 channels
        X_train_full = np.expand_dims(X_train_full, axis=-1)
        X_test = np.expand_dims(X_test, axis=-1)
        
        # Use np.tile to get correct shape
        X_train_full = np.tile(X_train_full, (1, 1, 1, 3))
        X_test = np.tile(X_test, (1, 1, 1, 3))
    
    print("Training data shape:", X_train_full.shape)
    print("Test data shape:", X_test.shape)
    
    # Run training with both 5-fold and 10-fold CV
    metrics_df = train_coatnet(
        X_train_full, 
        y_train_full, 
        X_test, 
        y_test, 
        k_folds=[5, 10,15,20],
        epochs=50
    )
    
    # Print summary of results
    print("\nSummary of training results:")
    
    # Group by K and calculate mean metrics
    for k in [5, 10,15,20]:
        k_metrics = metrics_df[metrics_df['K'] == k]
        print(f"\n{k}-fold CV results:")
        print(f"Average Test Accuracy: {k_metrics['Test_Accuracy'].mean():.4f}")
        print(f"Average F1 Score: {k_metrics['F1_Score'].mean():.4f}")
        print(f"Average AUC-ROC: {k_metrics['AUC_ROC'].mean():.4f}")
        print(f"Average Training Time: {k_metrics['Time_Taken'].mean():.2f} seconds")
        print(f"Average Epochs Run: {k_metrics['Epochs_Run'].mean():.1f}")

Training data shape: (800, 224, 224, 3)
Test data shape: (200, 224, 224, 3)

=== Starting 5-fold cross-validation ===
Fold 1/5
Training fold shape: (640, 224, 224, 3)
Validation fold shape: (160, 224, 224, 3)
Epoch 1/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 92s 8s/step - AUC: 0.6144 - Precision: 0.6417 - Recall: 0.6388 - accuracy: 0.6286 - loss: 0.9631 - val_AUC: 0.7678 - val_Precision: 0.0000e+00 - val_Recall: 0.0000e+00 - val_accuracy: 0.5188 - val_loss: 0.6449 - learning_rate: 0.0010
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 89s 9s/step - AUC: 0.7173 - Precision: 0.6777 - Recall: 0.6814 - accuracy: 0.6697 - loss: 0.6582 - val_AUC: 0.7335 - val_Precision: 0.6018 - val_Recall: 0.8831 - val_accuracy: 0.6625 - val_loss: 0.6121 - learning_rate: 0.0010
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 86s 9s/step - AUC: 0.7646 - Precision: 0.6909 - Recall: 0.8140 - accuracy: 0.7297 - loss: 0.5680 - val_AUC: 0.7360 - val_Precision: 0.5530 - val_Recall: 0.9481 - val_accuracy: 0.6062 - val_loss: 0.6865 - learning_rate